In [2]:
import json
import numpy as np
import cv2
import heapq
import os
import networkx as nx
import time
from sklearn.neighbors import KDTree, BallTree
# from scipy.spatial import KDTree 
import torchvision
from PIL import Image
import torch
import yaml
import shapely.geometry as geom
import scipy
import matplotlib.pyplot as plt
from pos_regression_control import PosRegModel
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import pickle, random

ModuleNotFoundError: No module named 'pos_regression_control'

In [ ]:
# Load keypoints from JSON files in a given directory
def load_keypoints_from_json(directory):
    configurations = []
    configuration_ids = []
    for filename in os.listdir(directory):
        if filename.endswith('.json') and not filename.endswith('_joint_angles.json') and not filename.endswith('_vel.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                data = json.load(file)
                keypoints = [np.array(point[0][:2], dtype=int) for point in data['keypoints']]  # Extracting x, y coordinates
                
                configurations.append(np.array(keypoints))
                configuration_ids.append(data['id'])  # Store the configuration ID

    print("length of configurations", len(configurations))
    return configurations, configuration_ids

def load_joint_angles_from_json(directory):
    joint_angles_dict = {}
    for filename in os.listdir(directory):
        if filename.endswith('_joint_angles.json'):
            with open(os.path.join(directory, filename), 'r') as file:
                data = json.load(file)
                joint_angles_dict[data['id']] = np.array(data['joint_angles'])
    return joint_angles_dict

def load_keypoints_from_truncated_json(directory):
    configurations = []
    for filename in os.listdir(directory):
        # if filename.endswith('.json'):
        if filename.endswith('.json') and not filename.endswith('_joint_angles.json') and not filename.endswith('_vel.json'):
            file_index = int(filename.split('.')[0])
            if file_index >= 10000:
                file_path = os.path.join(directory, filename)
                with open(file_path, 'r') as file:
                    data = json.load(file)
                    # Convert keypoints to integers
                    keypoints = [np.array(point[0][:2], dtype=int) for point in data['keypoints']]  # Extracting x, y coordinates
                    configurations.append(np.array(keypoints))

    return configurations

def load_and_sample_configurations(directory, num_samples):
    # Load configurations from JSON files
    configurations = load_keypoints_from_json(directory)

    # If there are more configurations than needed, sample a subset
    if len(configurations) > num_samples:
        sampled_indices = np.random.choice(len(configurations), size=num_samples, replace=False)
        sampled_configurations = [configurations[i] for i in sampled_indices]
    else:
        sampled_configurations = configurations

    return sampled_configurations

In [3]:
def skip_configurations(configurations, configuration_ids, skip_step, start, end):
    # Skip the configurations based on the step, start, and end parameters
    skipped_configs = configurations[start:end:skip_step]
    skipped_ids = configuration_ids[start:end:skip_step]
    
    # Pair the skipped configurations with their IDs for consistent shuffling
    paired_configs = list(zip(skipped_configs, skipped_ids))
    
    # Shuffle the paired configurations and IDs
    random.shuffle(paired_configs)
    
    # Unzip the shuffled pairs back into configurations and IDs
    shuffled_configs, shuffled_ids = zip(*paired_configs)

    print("length of shuffled configurations", len(shuffled_configs))
    
    return list(shuffled_configs), list(shuffled_ids)

def load_model_for_inference(model_path):    
    model = PosRegModel(18)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to inference mode
    return model

def predict_custom_distance(current_config, next_config, model):
    # Convert to 2D tensors if necessary
    start_kp_flat = torch.tensor(current_config.flatten(), dtype=torch.float).unsqueeze(0)  # Add batch dimension
    next_kp_flat = torch.tensor(next_config.flatten(), dtype=torch.float).unsqueeze(0)  # Add batch dimension

    # Predict the next configuration
    with torch.no_grad():
        output = model(start_kp_flat, next_kp_flat).squeeze(0).numpy()  # Remove batch dimension for output

    distance = np.linalg.norm(output)
    return float(distance)  # Reshape to the original configuration format

def custom_distance(x, y):
    # Ensure x and y are in the format the model expects (flattened arrays)
    return predict_custom_distance(x, y, model)

In [4]:
def build_lazy_roadmap_with_kdtree(configurations, configuration_ids, joint_angles_dict, model, k_neighbors):
    """
    Build a LazyPRM roadmap using a KDTree for efficient nearest neighbor search.
    
    Args:
    - configurations: List[np.array], a list of configurations (keypoints flattened).
    - k_neighbors: int, the number of neighbors to connect to each node.
    
    Returns:
    - G: nx.Graph, the constructed roadmap.
    """

    flattened_configs = np.vstack([config.flatten() for config in configurations])
    # Extract joint angles using the configuration IDs
    joint_angles_list = [joint_angles_dict[config_id] for config_id in configuration_ids]
    joint_angles_array = np.vstack(joint_angles_list)

    custom_tree = BallTree(flattened_configs, metric=custom_distance)
    euclidean_tree = KDTree(flattened_configs)
    gt_tree = KDTree(joint_angles_array)

    custom_G = nx.Graph()
    euclidean_G = nx.Graph()
    gt_G = nx.Graph()

    # Add nodes in the graph from the configurations and 
    for i, (config, config_id) in enumerate(zip(configurations, configuration_ids)):
        custom_G.add_node(i, configuration=config, joint_angles=joint_angles_dict[config_id])
        euclidean_G.add_node(i, configuration=config, joint_angles=joint_angles_dict[config_id])
        gt_G.add_node(i, configuration=config, joint_angles=joint_angles_dict[config_id])        

    # generate nearest neighbors with custom joint distance
    for i, config in enumerate(flattened_configs):
        print('node_', i)
        distances, indices = custom_tree.query([config], k=k_neighbors + 1)  # +1 to include self in results
        for j, d in zip(indices[0], distances[0]):  # Skip self
            print('node_', j)
            if j != i:
                custom_G.add_edge(i, j, weight=d)
    print("Edges Custom", custom_G.edges)

    # generate nearest neighbors with default euclidean distances between configs in image space
    for i, config in enumerate(flattened_configs):
        print('node_', i)
        distances, indices = euclidean_tree.query([config], k=k_neighbors + 1)  # +1 to include self in results
        for j, d in zip(indices[0], distances[0]):  # Skip self
            print('node_', j)
            if j != i:
                euclidean_G.add_edge(i, j, weight=d)
    print("Edges Euclidean", euclidean_G.edges)

    # generate nearest neighbors with actual joint distances between configs in configuration space
    for i, joint_angles in enumerate(joint_angles_array):
        print('node_', i)
        distances, indices = gt_tree.query([joint_angles], k=k_neighbors + 1)  # +1 to include the node itself        
        for d, j in zip(distances[0], indices[0]):
            print('node_', j)
            if i != j:  # Avoid self-loops
                gt_G.add_edge(i, j, weight=d)  
    print("Edges GT", gt_G.edges)

    return custom_G, custom_tree, euclidean_G, euclidean_tree, gt_G, gt_tree

In [ ]:
directory = '/home/jc-merlab/Pictures/panda_data/panda_sim_vel/panda_rearranged_data/path_planning_rearranged/'  # Replace with the path to your JSON files
model_path = '/home/jc-merlab/Pictures/Data/trained_models/reg_pos_b128_e500_v32.pth'
custom_graph_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/custom_roadmap_angle_shuf_10_n3.pkl'
custom_tree_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/custom_tree_angle_shuf_10_n3.pkl'
euclidean_g_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/euclidean_roadmap_angle_shuf_10_n3.pkl'
euclidean_tree_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/euclidean_tree_angle_shuf_10_n3.pkl'
gt_graph_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/joint_space_roadmap_angle_shuf_10_n3.pkl'
gt_tree_path = '/home/jc-merlab/Pictures/Dl_Exps/sim_vs/servoing/configurations_and_goals/joint_space_tree_angle_shuf_10_n3.pkl'

configurations, configuration_ids = load_keypoints_from_json(directory)
model = load_model_for_inference(model_path)
joint_angles_dict = load_joint_angles_from_json(directory)

skip_step = 2500
start_index = 1
end_index = 25000

skipped_configs, skipped_ids = skip_configurations(configurations, configuration_ids, \
                                                       skip_step, start_index, end_index)

num_neighbors = 3

custom_roadmap, custom_tree, euclidean_roadmap, euclidean_tree, gt_roadmap, gt_tree = build_lazy_roadmap_with_kdtree(\
                                skipped_configs, skipped_ids, joint_angles_dict, model, num_neighbors)

In [ ]:
save_graph(custom_roadmap, custom_tree, euclidean_roadmap, euclidean_tree, \
                gt_roadmap, gt_tree, custom_graph_path, custom_tree_path, \
                euclidean_g_path, euclidean_tree_path, gt_graph_path, gt_tree_path)